# Output correction

In [5]:
import floret
model = floret.load_model("/home/zoltanvarju/PycharmProjects/colab_pipelines/models/lms/hufloret_.bin")

In [13]:
import os

data_path = "../data/processed/speech_segments"
text_path = "../data/processed/text_segments"

transcriptions = [os.path.join(data_path, f) for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]
for f in transcriptions:
    fname = f.split("/")[-1]
    with open(os.path.join(text_path, fname), "w") as outfile:
        with open(f, "r") as infile:
            transcript = infile.read().split()
        corrected = []
        for wd in transcript:
            if wd not in model.words:
                suggestion = model.get_nearest_neighbors(wd, 1)[0][1]
                corrected.append(suggestion)
            else:
                corrected.append(wd)
        outfile.write(" ".join(corrected))

# Full text from segments

In [24]:
out_path = "../data/processed/transcribed_programmes"

segments = [os.path.join(text_path, f) for f in os.listdir(text_path) if os.path.isfile(os.path.join(text_path, f))]

programmes = set([e.split("/")[-1].split("_")[0] for e in segments])

for programme in programmes:
    # TODO: ints in fname should be same length so we can simply sort them
    programme_segments = sorted([e for e in segments if programme in e])
    full_text = []
    for programme_segment in programme_segments:
        with open(programme_segment, "r") as infile:
            full_text.append(infile.read().strip())
    fname = os.path.join(out_path, programme + ".txt")
    with open(fname, "w") as outfile:
        outfile.write("\n".join(full_text))